# Interpreter data

This is a quite short file, which is good. This is pretty easy to analyse at the start, there's a signature, some constant, and a count value with corresponds to a number of records seemingly named `*.gw` and `*.gs`.

In [1]:
from pathlib import Path
from struct import unpack_from

path = Path("install") / "v1.0-us-pre" / "zbd" / "interp.zbd"

data = path.read_bytes()
signature, seven, count = unpack_from("<3I", data, 0)

# these are always the same?
assert signature == 0x08971119, signature
assert seven == 7, seven

offsets = {}
unknown = {}
for i in range(count):
    raw_name, unk1, unk2, unk3, unk4, offset = unpack_from(
        "<120s4BI", data, 3 * 4 + i * 128
    )
    assert unk4 in (53, 54, 55)
    name = raw_name.rstrip(b"\0").decode("ascii")
    offsets[name] = offset
    unknown[name] = (unk1, unk2, unk3, unk4)

# are the records ordered?
assert list(offsets.values()) == sorted(offsets.values())

This works rather well. Of note is that the lengths of the data isn't stored. This would make sense if the data is self-terminating, and the program just jumps to the offset and executes the script, although it's still interesting that memory didn't need to be known in advance. Maybe the unknown values signify how many objects will be created? It doesn't seem to matter for extraction.

Parsing the script is quite easy, which makes sense for a 90s game. It is somewhat interesting that the operations are stored as strings. The operation "size" is first, followed by the argument count (which includes the operation name), and then the entire operation and it's arguments in string-form, delimited with a zero-character.

In [2]:
iter_offset = iter(offsets.values())
script_data = data[next(iter_offset) : next(iter_offset)]

offset = 0
script = []
while True:
    (size,) = unpack_from("<I", script_data, offset)
    offset += 4

    # end of script
    if size == 0:
        break

    (arg_count,) = unpack_from("<I", script_data, offset)
    offset += 4

    operation = script_data[offset : offset + size].decode("ascii")
    offset += size
    assert arg_count == operation.count("\0")
    assert operation.endswith("\0")
    script.append(operation.rstrip("\0"))

Time to put it all together then. I'm also going to dump out all the operations, to make it easy to see which ones need implementing.

In [3]:
import json


operations = set()


def parse_script(data, offset):
    script = []
    while True:
        (size,) = unpack_from("<I", data, offset)
        offset += 4

        # end of script
        if size == 0:
            break

        (arg_count,) = unpack_from("<I", data, offset)
        offset += 4

        operation = data[offset : offset + size].decode("ascii")
        offset += size
        assert arg_count == operation.count("\0")
        assert operation.endswith("\0")
        script.append(operation.rstrip("\0"))
        operations.add((operation.split("\0")[0], arg_count - 1))

    return script


def parse_interp(data):
    signature, seven, count = unpack_from("<3I", data, 0)
    assert signature == 0x08971119, signature
    assert seven == 7, seven

    scripts = {}
    for i in range(count):
        raw_name, offset = unpack_from("<120s4xI", data, 3 * 4 + i * 128)
        name = raw_name.rstrip(b"\0").decode("ascii")
        scripts[name] = parse_script(data, offset)
    return scripts


base_path = Path.cwd() / "interp"
for path in Path("install").rglob("interp.zbd"):
    json_path = base_path / f"{path.parent.parent.name}.json"
    with json_path.open("w", encoding="utf-8") as f:
        json.dump(parse_interp(path.read_bytes()), f, indent=2)

json_path = base_path / "operations.json"
with json_path.open("w", encoding="utf-8") as f:
    json.dump(sorted(operations, key=lambda op: op[0]), f, indent=2)

Well, that was nice and easy for once. Of course, there's a lot of interesting information in these files. And for implementing an interpreter, this would just be the beginning. Still, this is all that's needed for preservation, so I'll move on.

## Next up

Next up would be the `anim.zbd`, or `gamez.zbd` files. However, I haven't made too much progress on these yet. This is a shame, it would be great to be able to extract the maps. Stay tuned!